In [1]:
import numpy as np
import tensorflow as tf

In [2]:
def khatrirao_product(tensor1, tensor2):
    i0 = tf.constant(1)
    prod0 = tf.multiply(tensor1[0, :], tensor2)
    _, khprod = tf.while_loop(lambda i, m: tf.less(i, tf.shape(tensor1)[0]),
                              lambda i, m: [i+1, tf.concat([m, tf.multiply(tensor1[i, :], tensor2)], 0)],
                              loop_vars=[i0, prod0],
                              shape_invariants=[i0.get_shape(), tf.TensorShape([None, None])])
    return khprod

In [3]:
def rank3tensor_decomposition_ALS(matrix, k, alpha=0.01, nbiter=1000):
    dim0, dim1, dim2 = matrix.shape

    X = tf.placeholder(tf.float32, shape=(None, None, None), name='X')
    #X_shape = tf.shape(X)
    #Xdim0, Xdim1, Xdim2 = X_shape[0], X_shape[1], X_shape[2]
    
    X0 = tf.reshape(tf.transpose(X, perm=(0, 2, 1)), shape=(dim0, dim1*dim2))
    X1 = tf.reshape(tf.transpose(X, perm=(1, 2, 0)), shape=(dim1, dim0*dim2))
    X2 = tf.reshape(tf.transpose(X, perm=(2, 0, 1)), shape=(dim2, dim0*dim1))
    
    A = tf.Variable(initial_value=tf.random_normal([dim0, k]), name='A')
    B = tf.Variable(initial_value=tf.random_normal([dim1, k]), name='B')
    C = tf.Variable(initial_value=tf.random_normal([dim2, k]), name='C')

    costA = tf.reduce_sum(tf.abs(X0 - tf.matmul(A, tf.transpose(khatrirao_product(C, B)))))
    costB = tf.reduce_sum(tf.abs(X1 - tf.matmul(B, tf.transpose(khatrirao_product(C, A)))))
    costC = tf.reduce_sum(tf.abs(X2 - tf.matmul(C, tf.transpose(khatrirao_product(B, A)))))
    
    trainA = tf.train.GradientDescentOptimizer(alpha).minimize(costA)
    trainB = tf.train.GradientDescentOptimizer(alpha).minimize(costB)
    trainC = tf.train.GradientDescentOptimizer(alpha).minimize(costC)
    
    init = tf.global_variables_initializer()
    
    sess = tf.Session()
    sess.run(init)

    for _ in range(nbiter):
        sess.run(trainA, feed_dict={X: x})
        sess.run(trainB, feed_dict={X: x})
        sess.run(trainC, feed_dict={X: x})

    return sess.run((A, B, C), feed_dict={X: x})

# Test

In [4]:
x1 = np.array([[1, 4, 7, 10], [2, 5, 8, 11], [3, 6, 9, 12]], dtype=np.float32)
x2 = np.array([[13, 16, 19, 22], [14, 17, 20, 23], [15, 18, 21, 24]], dtype=np.float32)
x = np.zeros((3, 4, 2))
x[:, :, 0] = x1
x[:, :, 1] = x2

In [5]:
A, B, C = rank3tensor_decomposition_ALS(x, k=5, nbiter=10000)

In [6]:
mat = np.zeros((A.shape[0], B.shape[0], C.shape[0]))

In [7]:
from itertools import product

for i, j, k in product(range(A.shape[0]), range(B.shape[0]), range(C.shape[0])):
    mat[i, j, k] = sum(A[i, alpha]*B[j, alpha]*C[k, alpha] for alpha in range(5))

In [8]:
mat

array([[[ 1.10114574, 13.09981078],
        [ 3.66990714, 15.46098259],
        [ 6.44580641, 18.98348343],
        [ 7.73837258, 19.43592691]],

       [[ 2.22968877, 15.32210699],
        [ 3.87107951, 16.50796902],
        [ 7.95512927, 21.52198064],
        [ 9.78726657, 21.99262214]],

       [[ 3.13514873, 15.4740954 ],
        [ 4.96382653, 16.6664874 ],
        [ 7.34533671, 19.52429676],
        [10.88494436, 22.1497407 ]]])

x

In [9]:
x

array([[[ 1., 13.],
        [ 4., 16.],
        [ 7., 19.],
        [10., 22.]],

       [[ 2., 14.],
        [ 5., 17.],
        [ 8., 20.],
        [11., 23.]],

       [[ 3., 15.],
        [ 6., 18.],
        [ 9., 21.],
        [12., 24.]]])